<a href="https://colab.research.google.com/github/travismsanders687/practical-pytorch/blob/master/user_features_table_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

#Hello

In [ ]:
# Commonly used modules
import numpy as np
import datetime
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import cv2
import IPython
from pandas_profiling import ProfileReport
from six.moves import urllib

# GCP
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
project_id = "nomad-prod-193619"
%load_ext google.colab.data_table

client = bigquery.Client(project=project_id)

dataset = client.query('''
  WITH
  submission_date AS (
  SELECT
    MIN(ps.timestamp) AS first_submission_timestamp,
    p._id,
    ps.status,
    p.physician,
    p.jobpost
  FROM
    `nomad-prod-193619.ml_team.placement_unique_view` p
  LEFT JOIN
    UNNEST(p.placement_status_history) ps
  WHERE
    status IN ("submitted_to_vms",
      "packet_submitted",
      "submitted_packet")
  GROUP BY
    2,
    3,
    4,
    5 )
SELECT
  job_post.source AS VMS,
  placement.application.has_curriculum_vitae,
  placement.application.years_of_nursing_experience,
  placement.application.years_of_travel_experience,
  user.age_group,
  user.num_qualified_references,
  user._id as user_id,
  placement.application_submission_date as application_submission_date,
  ARRAY_LENGTH(placement.application.work_history) AS work_history_entries,
  ARRAY_LENGTH(placement.application.generic_certifications) AS generic_certifications,
  DATETIME_DIFF(DATETIME(placement.application_submission_date),
    DATETIME(job_post.date_first_published),
    DAY) AS publish_to_submission_diff,
  DATETIME_DIFF( DATETIME(placement.application_submission_date),
    DATETIME(job_post.start_date),
    DAY) AS start_date_diff,
  submission_date.first_submission_timestamp AS first_submission_timestamp,
  COUNT(DISTINCT prophecy._id) AS count_completed_assessments,
    --placement rollup fields
  num_previous_applications,
  has_previous_packet_created,
  has_previous_submission,
  has_previous_offer,
  has_previous_hire,
  days_since_last_packet_created,
  days_since_last_packet_submission,
  days_since_last_offer,
  days_since_last_hire,
  case when days_since_last_packet_created < 30 then 1 else 0 end as has_packet_created_30_days,
  case when days_since_last_packet_submission < 30 then 1 else 0 end as has_submission_30_days
FROM
  `nomad-prod-193619.ml_team.placement_unique_view` placement
INNER JOIN
  `nomad-prod-193619.ml_team.user_unique_view` user
ON
  placement.physician = user._id
INNER JOIN
  `nomad-prod-193619.ml_team.job_post_unique_view` job_post
ON
  placement.jobpost = job_post._id
INNER JOIN
  `nomad-prod-193619.ml_team.placement_rollup_temporal` placement_rollup
ON
  placement_rollup._id = placement._id
LEFT JOIN
  submission_date
ON
  placement._id = submission_date._id
LEFT JOIN `nomad-prod-193619.ml_team.prophecy_assessment_unique_view` prophecy
ON
  prophecy.provider = user._id 
AND prophecy.date_completed <= placement.application_submission_date
WHERE
  placement.application_submission_date IS NOT NULL
  AND job_post.job_type = "travel"
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23
ORDER BY
  submission_date.first_submission_timestamp ASC
''').to_dataframe()




ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: ignored

In [ ]:
dataset.dtypes

VMS                                               object
has_curriculum_vitae                              object
years_of_nursing_experience                      float64
years_of_travel_experience                       float64
age_group                                         object
num_qualified_references                           int64
user_id                                           object
application_submission_date          datetime64[ns, UTC]
work_history_entries                               int64
generic_certifications                             int64
publish_to_submission_diff                         int64
start_date_diff                                  float64
first_submission_timestamp           datetime64[ns, UTC]
count_completed_assessments                        int64
num_previous_applications                          int64
has_previous_packet_created                        int64
has_previous_submission                            int64
has_previous_offer             

[Spreadsheet](https://docs.google.com/spreadsheets/d/1TvgPxnBVgELN7WDbH5MN-XMLF9DibNGbVqaWyNh8AWs/edit?ts=5ea0b6f4#gid=0)

| variable 	| dtype 	|
|-	|-	|
| vms 	| string/categorical 	|
| has_cv 	| boolean 	|
| years_of_nursing_experience 	| float (int) 	|
| years_of_travel_experience 	| float (int) 	|
| age_group 	| ordinal 	|
| num_qualified_references 	| int 	|
| num_previous_applications 	| int 	|
| has_previous_packet_created 	| boolean 	|
| has_previous_packet_submission 	| boolean 	|
| has_previous_offer 	| boolean 	|
| has_previous_hire 	| boolean 	|
| work_history_entries 	| int 	|
| generic_certifications 	| int 	|
| published_to_application_submission_diff 	| int 	|
| start_date_diff 	| int 	|
| has_packet_created_30_days 	| boolean 	|
| has_submission_30_days 	| boolean 	|
| days_since_last_packet_created 	| int 	|
| days_since_last_packet_submission 	| int 	|
| days_since_last_offer 	| int 	|
| days_since_last_hire 	| int 	|
| previous packet created for this VMS 	| boolean 	|

In [ ]:
bool_cols = [col for col in dataset.columns if col.find("has") > -1 or col.find("binary") > -1]
ts_cols = [col for col in dataset.columns if col.endswith("date") or col.endswith("timestamp")]

In [ ]:
## needs to be a reusable function
for col in bool_cols:
    if isinstance(dataset.dtypes[col], int):
        dataset[col] = bool(dataset[col])
    else:
        dataset[col] = dataset[col].apply(lambda x: True if x == "True" else False)
for col in ts_cols:
    dataset[col] = pd.to_datetime(dataset[col])

In [ ]:
dataset.dtypes

VMS                                               object
has_curriculum_vitae                                bool
years_of_nursing_experience                      float64
years_of_travel_experience                       float64
age_group                                         object
num_qualified_references                           int64
user_id                                           object
application_submission_date          datetime64[ns, UTC]
work_history_entries                               int64
generic_certifications                             int64
publish_to_submission_diff                         int64
start_date_diff                                  float64
first_submission_timestamp           datetime64[ns, UTC]
count_completed_assessments                        int64
num_previous_applications                          int64
has_previous_packet_created                         bool
has_previous_submission                             bool
has_previous_offer             

In [ ]:
## convert object columns to string
for col in dataset.columns:
    if repr(dataset.dtypes[col]) == "dtype('O')":
        dataset[col] = dataset[col].astype("|S")

In [ ]:
## need to moved to ./lib/preprocessing.py
def add_vms_submission_binary(data):
    data["vms_submission_binary"] = data["first_submission_timestamp"].apply(lambda x: 0 if pd.isnull(x) else 1)
    return data


def compute_hours_since_last_packet_created_all(data):
    use = data[["user_id", "first_submission_timestamp"]].copy()
    data["hours_since_last_packet_created"] = use.groupby("user_id")["first_submission_timestamp"].diff()
    return data  


def compute_user_has_packet_for_vms_v2(data):
    local_data = data[["user_id", "VMS", "application_submission_date", "first_submission_timestamp"]].copy()
    previous_vms_submissions = pd.DataFrame(columns=["previous_vms_submission"])
    idx = list(local_data.columns)
    application_submission_date_idx = idx.index("application_submission_date") + 1 #index of df is at 0
    user_id_idx = idx.index("user_id") + 1
    vms_idx = idx.index("VMS") + 1
    for row in local_data.itertuples():
        submission_match = local_data["first_submission_timestamp"] <= row[application_submission_date_idx]
        user_id_match = row[user_id_idx] == local_data["user_id"]
        vms_match = row[vms_idx] == local_data["VMS"]
        previous_vms_submissions.loc[row[0]] = not local_data[submission_match].empty

    data["previous_vms_submission"] = previous_vms_submissions["previous_vms_submission"]
    return data

dataset = add_vms_submission_binary(dataset)
dataset = compute_hours_since_last_packet_created_all(dataset)
# dataset = compute_user_has_packet_for_vms_v2(dataset)

## running the below line is really slow
## uncomment on updates to dataset, if interested
# ProfileReport(dataset)

In [ ]:
from sklearn import preprocessing

## need to moved to ./lib/preprocessing.py
def encode_categorical(data, categories, encoding="one_hot"):
    ## categories is a list
    if encoding == "one_hot":
        enc = preprocessing.OneHotEncoder()
    elif encoding == "ordinal":
        enc = preprocessing.OrdinalEncoder()
    else:
      ## throw err, unsupported encoding
        pass
    enc.fit(categories)
    if enc == "one_hot":
        return enc.transform(data.values.reshape(-1, 1)).toarray()
    elif enc == "ordinal":
        return enc.transform(data.values.reshape(-1, 1))


In [ ]:
%time
dataset = add_vms_submission_binary(dataset)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [ ]:
%time
dataset = compute_hours_since_last_packet_created_all(dataset)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [ ]:
vms_one_hot = encode_categorical(dataset["VMS"], categories=[[vms] for vms in dataset["VMS"].unique()])

vms_proc = pd.DataFrame(vms_one_hot, columns = ["vms_"+str(int(i)) for i in range(len(dataset["VMS"].unique()))]) 

dataset = pd.concat([dataset, vms_proc], axis=1)

In [ ]:
dataset["hours_since_last_packet_created"] = dataset["hours_since_last_packet_created"] / datetime.timedelta(hours=1)

In [ ]:
X = dataset.copy()
X = X.drop(
    columns=[
        "user_id",
        "application_submission_date",
        "first_submission_timestamp",
        "vms_submission_binary",
        "VMS",
        "age_group"
    ]
)
y = dataset.vms_submission_binary.copy()

In [ ]:
## gotta fix this...
dataset = dataset.fillna(0)

In [ ]:
X.head()

,has_curriculum_vitae,years_of_nursing_experience,years_of_travel_experience,num_qualified_references,work_history_entries,generic_certifications,publish_to_submission_diff,start_date_diff,count_completed_assessments,num_previous_applications,has_previous_packet_created,has_previous_submission,has_previous_offer,has_previous_hire,days_since_last_packet_created,days_since_last_packet_submission,days_since_last_offer,days_since_last_hire,has_packet_created_30_days,has_submission_30_days,hours_since_last_packet_created,vms_0,vms_1,vms_2,vms_3,vms_4,vms_5,vms_6,vms_7,vms_8,vms_9,vms_10,vms_11,vms_12,vms_13,vms_14,vms_15,vms_16,vms_17,vms_18,vms_19,vms_20,vms_21,vms_22,vms_23,vms_24,vms_25,vms_26,vms_27,vms_28,vms_29,vms_30
0,False,10.0,2.0,3,3,1,2,-24.0,0,0,False,False,False,False,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,40.0,0.0,0,3,2,36,NaN,0,11,False,False,False,False,1.0,0.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,6.0,4.0,3,11,5,2,-15.0,2,35,False,False,False,False,2.0,2.0,102.0,102.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,5.0,3.0,3,10,5,6,-19.0,1,30,False,False,False,False,0.0,1.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,5.0,3.0,3,10,5,0,-13.0,1,29,False,False,False,False,0.0,0.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
y.describe()

count    60968.000000
mean         0.311754
std          0.463214
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: vms_submission_binary, dtype: float64


Need to write `X` this to file or BigQuery and preserve dtypes, so we can be explicit on loading them in other notebook and not have to recast everything


In [ ]:
dataset_id = 'ml_team'
table_id = 'preprocessed_data'
table_reference = client.dataset(dataset_id).table(table_id)
client.delete_table(table_reference, not_found_ok=True)

dataset.to_gbq(
    'ml_team.preprocessed_data',
    'nomad-prod-193619',
    chunksize=10000,
)


7it [00:40,  5.84s/it]


#Hello